In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")



import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

#from tqdm import tqdm
import os

In [2]:

# using the SQLite Table to read data.
con = sqlite3.connect('database.sqlite') 

#1 2 as negative
#4 5 positive
#ignore

filtered_data = pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score != 3""", con) 

# Give reviews with Score>3 a positive rating, and reviews with a score<3 a negative rating.
def partition(x):
    if x < 3:
        return 'negative'
    return 'positive'

#changing reviews with score less than 3 to be positive and vice-versa
actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition) 
filtered_data['Score'] = positiveNegative
print("Number of data points in our data", filtered_data.shape)
filtered_data.head(3)
#Time unique time stamp format

Number of data points in our data (525814, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...


In [35]:
display = pd.read_sql_query("""
SELECT * FROM Reviews 
WHERE Score != 3 AND UserID = "AR5J8UI46CURR" 
ORDER BY ProductID 
""",con)
display
#Deduplication

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


In [3]:
#sorting the data according to the productid
sorted_data = filtered_data.sort_values('ProductId',axis=0,ascending=True)
sorted_data.shape

(525814, 10)

In [4]:
#Deduplication
final= sorted_data.drop_duplicates(subset={'UserId','ProfileName','Time','Text'},keep='first',inplace=False)


In [5]:
final.shape

(364173, 10)

In [6]:
#Remaining Data
(final['Id'].size)/(sorted_data['Id'].size)*100

69.25890143662969

In [7]:
#Onemore scenario Numerator always should be less than denominator
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]
#We wil remove this observation comes when we observe more times data

In [8]:
final.shape

(364171, 10)

In [9]:
final['Score'].value_counts()

positive    307061
negative     57110
Name: Score, dtype: int64

#7.2.2 Text Preprocessing Stemming Stopword Removal and Lemmatizaton
=======================================

#BOW
===

In [10]:
count_vect = CountVectorizer()
final_counts= count_vect.fit_transform(final['Text'].values)

In [11]:
type(final_counts) #Same as sparse matrix contains 0s and 1s

scipy.sparse.csr.csr_matrix

In [12]:
final_counts.get_shape() #BOW Representation

(364171, 115281)

In [13]:
#Find sentences containing HTML Tags
i=0;
for sent in final['Text'].values:
    if(len(re.findall('<.*?>',sent))):
        print(i)
        print(sent)
        break;
    i +=1;

6
I set aside at least an hour each day to read to my son (3 y/o). At this point, I consider myself a connoisseur of children's books and this is one of the best. Santa Clause put this under the tree. Since then, we've read it perpetually and he loves it.<br /><br />First, this book taught him the months of the year.<br /><br />Second, it's a pleasure to read. Well suited to 1.5 y/o old to 4+.<br /><br />Very few children's books are worth owning. Most should be borrowed from the library. This book, however, deserves a permanent spot on your shelf. Sendak's best.


In [85]:
nltk.download()#To download stopwords

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [14]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer



In [15]:
stop= set(stopwords.words('english')) #Set of Stopwords

In [16]:
sno = nltk.stem.SnowballStemmer('english') #To initialise Snowball Stemmer

In [17]:
#https://pymotw.com/2/re/ Refer
#To find the HTML tag and replace the data with Space
def cleanhtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr , ' ' , sentence)
    return cleantext

In [18]:
#To Clean Punctuation(any) in the sentence
def cleanpunc(sentence):
    cleaned= re.sub(r'[? | ! | \' | # | " ]' , r'',sentence)
    cleaned= re.sub(r'[.|,|(|)|\|/]',r'',cleaned)
    return cleaned

In [92]:
print(stop) #To view all the stopwords

{'doing', 'mustn', 'should', 'herself', 'yourself', 'she', 'other', 'but', 'now', 'just', 'whom', 'to', 're', "needn't", 'does', 'below', 'for', "didn't", 'than', 'ours', 'you', "she's", 'there', 'until', "hasn't", 'themselves', 'we', 'its', 'd', 'it', "it's", 'which', 'wasn', 'couldn', "wouldn't", 'is', 'of', 'after', 'myself', "wasn't", 'won', 'wouldn', 'y', 'm', 'her', 't', 'itself', 'his', 'again', "you've", 'no', 'will', 'on', 'needn', 'so', 'was', "you're", 'before', 'and', 'from', 'our', 'during', "isn't", "should've", 'theirs', 'such', "haven't", 'o', 'their', 'being', 'between', "you'd", 'what', 'hadn', 'not', 'above', "hadn't", 've', 'too', 'under', 'about', "that'll", 'where', 'an', 'only', 'very', 'ourselves', 'yours', 'into', 'them', 'this', 'out', 'having', "mightn't", 'then', 'himself', 'haven', 'yourselves', "don't", 'same', "shan't", 'aren', "weren't", 'they', 'shouldn', 's', 'll', 'at', 'has', "shouldn't", 'here', 'when', 'had', 'with', 'are', 'be', "you'll", 'those',

In [93]:
print(sno.stem('tasty')) #To find Stem word

tasti


In [19]:
#Takes 10mins to execute
i=0
str1=''
final_string=[]
all_positive_words=[]#Store words realted to positive reviews here
all_negative_words=[]#Store words realted to negative reviews here

s=''

for sent in final['Text'].values:
    filtered_sentence=[]
    #print(sent)
    
    sent = cleanhtml(sent) #To remove HTML
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):
                if(cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s) #all filtered words +ve and -ve
                    if(final['Score'].values)[i] == 'positive':
                        all_positive_words.append(s) #List of all positive words'
                    if(final['Score'].values)[i] == 'negative':
                        all_negative_words.append(s) #list of all negative words
                else:
                    continue
            else:
                continue
    #print(filtered_sentence)
    str1=b"".join(filtered_sentence) #Final Filetered sentence
    final_string.append(str1)
    i+=1

In [21]:
#Freuqnecy Distribution of positive and Negative words
#Sugg unigram and bi gram
freq_dist_positive = nltk.FreqDist(all_positive_words)
freq_dist_negative = nltk.FreqDist(all_negative_words)

print("Most Common Positive Words: ",freq_dist_positive.most_common(20))
print("Most Common Negative Words: ",freq_dist_negative.most_common(20))

Most Common Positive Words:  [(b'like', 139072), (b'tast', 128077), (b'good', 112017), (b'flavor', 108653), (b'love', 107018), (b'use', 103603), (b'great', 103095), (b'one', 96537), (b'product', 90273), (b'tri', 86411), (b'tea', 83290), (b'coffe', 77985), (b'make', 74914), (b'get', 72029), (b'food', 64244), (b'would', 55457), (b'time', 54848), (b'buy', 54031), (b'realli', 52611), (b'eat', 51790)]
Most Common Negative Words:  [(b'tast', 34300), (b'like', 32225), (b'product', 28003), (b'one', 20521), (b'flavor', 19368), (b'would', 17947), (b'tri', 17691), (b'use', 15263), (b'good', 14908), (b'coffe', 14579), (b'get', 13770), (b'buy', 13711), (b'order', 12832), (b'food', 12643), (b'dont', 11808), (b'tea', 11574), (b'even', 11042), (b'box', 10734), (b'make', 9823), (b'time', 9720)]


If you observe like is the word present in the both reviews. i e like and not like are for positive and negative reviews but system conisdering both as like
we have to take care this by bigrams


In [22]:
#bi gram #tri gramm

count_vect = CountVectorizer(ngram_range=(1,2))
final_bigram_counts= count_vect.fit_transform(final['Text'].values)

In [23]:
final_bigram_counts.get_shape()
#massive increase in the shape of the resulted table. which is more complex

(364171, 2910192)

#TFIDF
===


In [24]:
tfidf_vect = TfidfVectorizer(ngram_range=(1,2))
final_bigram_counts= tfidf_vect.fit_transform(final['Text'].values)

In [26]:
final_bigram_counts.get_shape()

(364171, 2910192)

In [27]:
features=tfidf_vect.get_feature_names()
len(features)

2910192

In [31]:
features[100000:100020] #features

['ales until',
 'ales ve',
 'ales would',
 'ales you',
 'alessandra',
 'alessandra ambrosia',
 'alessi',
 'alessi added',
 'alessi also',
 'alessi and',
 'alessi are',
 'alessi at',
 'alessi brand',
 'alessi breadsticks',
 'alessi caffe',
 'alessi cento',
 'alessi chicken',
 'alessi coarse',
 'alessi coffees',
 'alessi decaf']

In [36]:
print(final_bigram_counts[3,:].toarray()[0]) #To view the 3rd review after TFIDF #It contains TFIDF value for the each word

[ 0.  0.  0. ...,  0.  0.  0.]


In [40]:
def top_tfidf_feat(row,features,top_n=25):
    #top n tfidf valus in the row with words
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats=[(features[i],row[i]) for i in topn_ids]
    df=pd.DataFrame(top_feats)
    df.columns = ['Features' , 'tfidf']
    return df



In [42]:
top_tdidf = top_tfidf_feat(final_bigram_counts[2,:].toarray()[0],features,25)
top_tdidf

,Features,tfidf
0,learn,0.218213
1,which invent,0.206281
2,handmotions,0.206281
3,handmotions which,0.206281
4,invent for,0.206281
5,each poem,0.206281
6,poems throughout,0.206281
7,the handmotions,0.206281
8,learn their,0.199903
9,their months,0.199903


#Word2Vec:
======
Our own word2Vex


In [47]:
#5mins
import gensim
i=0
list_of_sent=[]

for sent in final['Text'].values:
    filtered_sentence = []
    sent = cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if cleaned_words.isalpha():
                filtered_sentence.append(cleaned_words.lower())
            else:
                continue
    list_of_sent.append(filtered_sentence)

In [48]:
print(final['Text'].values[0])
print('******************')
print(list_of_sent[0])

this witty little book makes my son laugh at loud. i recite it in the car as we're driving along and he always can sing the refrain. he's learned about whales, India, drooping roses:  i love all the new words this book  introduces and the silliness of it all.  this is a classic book i am  willing to bet my son will STILL be able to recite from memory when he is  in college
******************
['this', 'witty', 'little', 'book', 'makes', 'my', 'son', 'laugh', 'at', 'loud', 'i', 'recite', 'it', 'in', 'the', 'car', 'as', 'were', 'driving', 'along', 'and', 'he', 'always', 'can', 'sing', 'the', 'refrain', 'hes', 'learned', 'about', 'whales', 'india', 'drooping', 'i', 'love', 'all', 'the', 'new', 'words', 'this', 'book', 'introduces', 'and', 'the', 'silliness', 'of', 'it', 'all', 'this', 'is', 'a', 'classic', 'book', 'i', 'am', 'willing', 'to', 'bet', 'my', 'son', 'will', 'still', 'be', 'able', 'to', 'recite', 'from', 'memory', 'when', 'he', 'is', 'in', 'college']


In [ ]:
#takes time of 
w2v_model = gensim.models.Word2Vec(list_of_sent,min_count=5,size=50,workers=4)
#min_count = no of times word should present to create d dim if the word not present mentioned no of times then d dim vec will not be creared


In [ ]:
words= list(w2v_model.wv.vocab)
print(len(words))#all words

In [ ]:
words= list(w2v_model.wv.vocab)

In [ ]:
w2v_model.wv.most_similar('tasty') #From our Data

In [ ]:
#list of words in BOW
count_vec_feats = count_vect.get_feature_names()
count_vec_feats.index('like')
count_vec_feats[0]

#AVg Word2Vec TFIDF W2V

In [ ]:
#Avg W2v
sent_vectirs = []
for sent in list_of_sent: #listpfsent contains all reviews out of it taking 1st review
    sent_vec = np.zeros(50)
    cnt_words=0;
    for word in sent: #review one
        try:
            vec=w2v_model.wv[word] #here for each word d dim vec will be created
            sent_vec += vec #Review vector all words D*word
            cnt_words += 1 #no of words in review
        except:
            pass
    sent_vec /= cnt_words
    sent_vectors.append(sent_vec)

print(len(sent_vectors))
print(len(sent_vectors[0]))

In [ ]:
#avg TFIDF w2v
tfidf_feat = tf_idf_vect.get_feature_names() #To geta ll TFIDF feature names

tfidf_sent_vectors =[];
row=0;

for sent in list_of_sent:
    sent_vec = np.zeros(50)
    weight_sum = 0;
    for word in sent:
        try:
            vec = w2v_model.wv[word] #Word will be to w2v
            tfidf = final_tf_idf[row,tfidf_feat.index(word)] #Finding theTFIDF of the word 
            sent_vec += (vec*tfidf) #multikplying the w2v with TFIDF
            weight_sum += tfidf #sum of all TFIDF
        except:
            pass
    sent_vec /= weight_sum #review/TFIDF SUM
    tfidf_sent_vectors.append(sent_vec)
    row +=1
